In [82]:
# auto reload notebook deps
%reload_ext autoreload
%autoreload 2

[autoreload of utils.dataset failed: Traceback (most recent call last):
  File "/usr/local/anaconda3/envs/spec-project/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/usr/local/anaconda3/envs/spec-project/lib/python3.8/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/usr/local/anaconda3/envs/spec-project/lib/python3.8/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/usr/local/anaconda3/envs/spec-project/lib/python3.8/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 604, in _exec
  File "<frozen importlib._bootstrap_external>", line 843, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/Users/anton164/git/fall-spec/utils/dataset.py", line 1, in <module>
    from utils import flatten
Import

In [83]:
import json
import pandas as pd

DATASETS = {
    "CentralParkNYC": {
        "input_location": "./data/tweet_objects/CentralParkNYC/2021-01-27-2021-02-06.json",
        "output_location": "./data/mstream_datasets/CentralParkNYC-2021-01-27-2021-02-06.json",
        "spike_start": "Feb 1, 2021, 17:00",
        "spike_end": "Feb 3, 2021, 04:00",
    },
    "united": {
        "input_location": "./data/tweet_objects/united/2020-12-05-2020-12-15.json",
        "output_location": "./data/mstream_datasets/united-2020-12-05-2020-12-15.json",
        "spike_start": "Dec 12, 2020, 05:00",
        "spike_end": "Dec 13, 2020, 06:00",
    },
    "nationalGridUS": {
        "input_location": "./data/tweet_objects/nationalGridUS/2020-10-01-2020-10-12.json",
        "output_location": "./data/mstream_datasets/nationalGridUS-2020-10-01-2020-10-12.json",
        "spike_start": "Oct 7, 2020, 17:00",
        "spike_end": "Oct 9, 2020, 04:00",
    }
}

selected_dataset = DATASETS["nationalGridUS"]
labeled_spike_start = pd.to_datetime(selected_dataset["spike_start"]).tz_localize("UTC")
labeled_spike_end = pd.to_datetime(selected_dataset["spike_end"]).tz_localize("UTC")

In [84]:
from utils.dataset import create_tweet_df

with open(selected_dataset["input_location"], "r") as f:
    raw_dataset = json.load(f)

df_tweets = create_tweet_df(raw_dataset["tweets"])

# Label tweets
df_tweets["is_anomaly"] = df_tweets.created_at.apply(
    lambda x: 0 if (x < labeled_spike_start or x > labeled_spike_end) else 1
)

df_tweets.head()

,id,text,created_at,hashtags,mentions,in_reply_to_user_id,user_id,retweet_count,quote_count,reply_count,like_count,retweeted,replied_to,quoted,is_anomaly
2125,1311464085071036416,@nationalgridus YESS!!! It’s back on,2020-10-01 00:32:58+00:00,[],[nationalgridus],109524559,812872943134982144,0,0,0,0,NaN,1311416336787091457,NaN,0
2124,1311464351459672065,@ToSaveEnergy @SCE @LimeEnergy @NYPAenergy @na...,2020-10-01 00:34:01+00:00,"[Maintenance, cooling, energyefficiency, refri...","[ToSaveEnergy, SCE, LimeEnergy, NYPAenergy, na...",17881345,1591198093,0,0,0,0,NaN,1311369549837676544,NaN,0
2123,1311464409559166976,RT @ToSaveEnergy: Another insightful #ActiveEf...,2020-10-01 00:34:15+00:00,[ActiveEfficiency],[ToSaveEnergy],None,1591198093,3,0,0,0,1311369549837676544,NaN,NaN,0
2122,1311469381868814337,@DeanFelicetti Hi Dean. We understand not havi...,2020-10-01 00:54:01+00:00,[],[DeanFelicetti],3225669874,109524559,0,0,0,0,NaN,1311404424661737477,NaN,0
2121,1311471757967134722,@DeanFelicetti @nationalgridus Didn’t the wind...,2020-10-01 01:03:27+00:00,[],"[DeanFelicetti, nationalgridus]",3225669874,170846420,0,0,0,0,NaN,1311404424661737477,NaN,0


In [85]:
# Export 
df_tweets.to_json(
    selected_dataset["output_location"],
    orient="records"
)

In [86]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2126 entries, 2125 to 0
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   id                   2126 non-null   object             
 1   text                 2126 non-null   object             
 2   created_at           2126 non-null   datetime64[ns, UTC]
 3   hashtags             2126 non-null   object             
 4   mentions             2126 non-null   object             
 5   in_reply_to_user_id  1302 non-null   object             
 6   user_id              2126 non-null   object             
 7   retweet_count        2126 non-null   int64              
 8   quote_count          2126 non-null   int64              
 9   reply_count          2126 non-null   int64              
 10  like_count           2126 non-null   int64              
 11  retweeted            416 non-null    object             
 12  replied_to          